# AWS Lambda

AWS Lambda is the Serverless product that AWS introduced at reInvent in November 2014.  Lambda has largely popularized the concept of Serverless and AWS continue to lead in this space.

AWS Lambda allows developers to create and deploy small functions and have the code executed without the need to provision servers.  You pay only for the resources used rather than for idle servers.

A Lambda function provides a handler which is the entry point for the Lambda written for the chosen runtime such as node.js, python etc.
A Lambda function accepts JSON-formatted input and will usually return the same.

Lambda function is invoked in response to an event, such as an S3 file upload, a change in a database table, a web request or a scheduled event.

Runtimes
The following languages, as well as custom runtimes, are now supported
- C#
- Go
- Java
- Node.js
- Python

## Closed-source

Unfortunately AWS Lambda itself if a proprietary service.

Nevertheless as the leader in this space it is important to understand it's capabilities and limitations.

There are many Open Source tools which allow to deploy to AWS Lambda such as
- Serverless(.com) - allows to deploy to several Serverless platforms
- Chalice (A Python Framework provided by AWS)
- Claudia (A Node.js Framework)

or even to emulate Lambda for testing purposes
- localstack
- docker-lambda (https://github.com/lambci/docker-lambda)

## Setup

To run these exercises you should first have
- installed the awscli package to provide the aws command and also
- configured either
  - the ~/.aws/configure file with your AWS account credentials or
  - created a sourceable ~/.aws/credentials.rc (can be in any location) file

```
> cat ~/.aws/credentials.rc

export AWS_ACCESS_KEY_ID="<your-access-key>"
export AWS_SECRET_ACCESS_KEY="<your-secret-access-key>"
export AWS_DEFAULT_REGION=us-west-1
```

If you have chosen to use an rc file, source it as ```source <your-aws-credentials-rc-file>```, e.g.

In [ ]:
. ~/.aws/credentials.rc

We can now use the aws cli utility to access lambda commands.

Let's investigate the available commands with ```aws lambda help```

In [59]:
aws lambda help 

LAMBDA()                                                              LAMBDA()



NAME
       lambda -

DESCRIPTION
          Overview

       This  is  the AWS Lambda API Reference . The AWS Lambda Developer Guide
       provides additional information. For the service overview, see What  is
       AWS  Lambda  , and for information about how the service works, see AWS
       Lambda: How it Works in the AWS Lambda Developer Guide .

AVAILABLE COMMANDS
       o add-layer-version-permission

       o add-permission

       o create-alias

       o create-event-source-mapping

       o create-function

       o delete-alias

       o delete-event-source-mapping

       o delete-function

       o delete-function-concurrency

       o delete-layer-version

       o get-account-settings

       o get-alias

       o get-event-source-mapping

       o get-function

       o get-function-configuration

       o get-layer-version

       o get-layer-version-policy

       o get-policy

      

Let's see if we have any functions defined already using the ```aws lambda list-functions``` command.
You won't have any functions if you just created your account.

In [60]:
aws lambda list-functions

{
    "Functions": []
}


Note that it is possible to create functions directly using the command ```aws lambda create-function``` but there exist several tools which facilitate function creation and deployment.

For information about use of ```aws lambda create-function``` refer to these articles
- https://www.tutorialspoint.com/aws_lambda/aws_lambda_creating_and_deploying_using_aws_cli.htm
- https://docs.aws.amazon.com/lambda/latest/dg/API_CreateFunction.html

We will continue using other frameworks.
- Chalice: A Python Serverless Micro-service framework for AWS Lambda
- Claudia: A Node.js Serverless framework for AWS Lambda
- Serverless: A Serverless platform for various platorms (AWS Lambda, Azure Functions, Google CloudFunctions and more)

## Chalice

Chalice is an open source project created by AWS, available here: https://github.com/aws/chalice

Documentation is at https://chalice.readthedocs.io/en/latest/

Chalice allows to
- deploy to a local test server
- deploy to AWS Lambda

Let's see what options are available using ```chalice --help``` command

In [62]:
chalice --help

Usage: chalice [OPTIONS] COMMAND [ARGS]...

Options:
  --version             Show the version and exit.
  --project-dir TEXT    The project directory.  Defaults to CWD
  --debug / --no-debug  Print debug logs to stderr.
  --help                Show this message and exit.

Commands:
  delete
  deploy
  gen-policy
  generate-pipeline  Generate a cloudformation template for a...
  generate-sdk
  invoke             Invoke the deployed lambda function NAME.
  local
  logs
  new-project
  package
  url


## Create a Chalice project

Let's start by creating a new Chalice project:

In [63]:
chalice new-project chalice-app
cd chalice-app
ls -al

total 20
drwxrwxr-x 3 user1 user1 4096 Jan 25 00:44 .
drwxrwxr-x 5 user1 user1 4096 Jan 25 00:44 ..
drwxrwxr-x 2 user1 user1 4096 Jan 25 00:44 .chalice
-rw-rw-r-- 1 user1 user1   37 Jan 25 00:44 .gitignore
-rw-rw-r-- 1 user1 user1  736 Jan 25 00:44 app.py
-rw-rw-r-- 1 user1 user1    0 Jan 25 00:44 requirements.txt


We see that Chalice had created a template project for us, looking at the app.py file we see that we have a Python application to run a REST API server with just one route for '/'

In [114]:
cat app.py

from chalice import Chalice

app = Chalice(app_name='0')


@app.route('/')
def index():
    return {'hello': 'world'}


# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
# @app.route('/hello/{name}')
# def hello_name(name):
#    # '/hello/james' -> {"hello": "james"}
#    return {'hello': name}
#
# @app.route('/users', methods=['POST'])
# def create_user():
#     # This is the JSON body the user sent in their POST request.
#     user_as_json = app.current_request.json_body
#     # We'll echo the json body back to the user in a 'user' key.
#     return {'user': user_as_json}
#
# See the README documentation for more examples.
#


## Run the function locally

Run the command ```chalice local``` to deploy a local test server

```> chalice local
Serving on http://127.0.0.1:8000
```

In [115]:
http 127.0.0.1:8000

HTTP/1.1 200 OK
Content-Length: 17
Content-Type: application/json
Date: Fri, 25 Jan 2019 01:33:24 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "hello": "world"
}



We see that chalice has allowed us to very quickly
- create an application skeleton and to
- test it locally.

### Updates

Try making a chance to to app.py file and saving it.

Notice that chalice detects the change and redeploys the server to take into account the changes:
```
user1@ip-172-31-21-116:~/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app$ chalice local
Serving on http://127.0.0.1:8000
127.0.0.1 - - [25/Jan/2019 00:45:04] "GET / HTTP/1.1" 200 -
Restarting local dev server.
Serving on http://127.0.0.1:8000
Restarting local dev server.
Serving on http://127.0.0.1:8000
```

## Deploying a function to AWS Lambda
Now let's deploy this to AWS Lambda using the ```chalice deploy``` command


In [66]:
chalice deploy --help

Usage: chalice deploy [OPTIONS]

Options:
  --autogen-policy / --no-autogen-policy
                                  Automatically generate IAM policy for app
                                  code.
  --profile TEXT                  Override profile at deploy time.
  --api-gateway-stage TEXT        Name of the API gateway stage to deploy to.
  --stage TEXT                    Name of the Chalice stage to deploy to.
                                  Specifying a new chalice stage will create
                                  an entirely new set of AWS resources.
  --connection-timeout INTEGER    Overrides the default botocore connection
                                  timeout.
  --help                          Show this message and exit.


In [67]:
aws lambda list-functions

{
    "Functions": []
}


In [68]:
chalice deploy

Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Creating lambda function: chalice-app-dev
Creating Rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev
  - Rest API URL: https://shet9854i1.execute-api.us-west-1.amazonaws.com/api/


In ust a few commands we've been able to create a project template, run a local test server and deploy our skeleton app to AWS Lambda.

Chalice greatly simplifies the deployment of Python functions.


In [69]:
aws lambda list-functions

{
    "Functions": [
        {
            "FunctionName": "chalice-app-dev",
            "FunctionArn": "arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev",
            "Runtime": "python3.6",
            "Role": "arn:aws:iam::568285458700:role/chalice-app-dev",
            "Handler": "app.app",
            "CodeSize": 10518,
            "Description": "",
            "Timeout": 60,
            "MemorySize": 128,
            "LastModified": "2019-01-25T00:46:57.627+0000",
            "CodeSha256": "7OxQgMx43v9fW7N1BL8O48Dn9jZerW9kqt6fycWru64=",
            "Version": "$LATEST",
            "VpcConfig": {
                "SubnetIds": [],
                "SecurityGroupIds": [],
                "VpcId": ""
            },
            "Environment": {
                "Variables": {}
            },
            "TracingConfig": {
                "Mode": "PassThrough"
            },
            "RevisionId": "e8b57e42-07d2-47db-91d8-27e6b714282f"
        }
    ]
}


Let's now invoke our deployed function, but if we didn't note the output of our deploy command how could we get the url, it is not provided by ```aws lambda list-functions```?

Chalice has the ```url``` command to determine the URL:

In [70]:
chalice url

https://shet9854i1.execute-api.us-west-1.amazonaws.com/api/


In [71]:
http $(chalice url)

HTTP/1.1 200 OK
Connection: keep-alive
Content-Length: 17
Content-Type: application/json
Date: Fri, 25 Jan 2019 00:47:21 GMT
Via: 1.1 d8f42fc9558e3e49ebfdf8834baeb756.cloudfront.net (CloudFront)
X-Amz-Cf-Id: msiOZ4amdmAft_uEIRj3jshOw5dT8zmfAv12w_Z_R1MMFcP3ZDm8iw==
X-Amzn-Trace-Id: Root=1-5c4a5c99-86603932d98ab306f458536d;Sampled=0
X-Cache: Miss from cloudfront
x-amz-apigw-id: UCNoBHizyK4FQTw=
x-amzn-RequestId: c634786e-203a-11e9-8a02-f1a3979b94fb

{
    "hello": "world"
}



## Cleanup: deleting the function deployment

We can cleanup by deleting the function which we deployed

In [72]:
aws lambda delete-function --function-name chalice-app-dev
aws lambda list-functions

{
    "Functions": []
}


# Function capabilities

In [128]:
cat > app.py << EOF
from chalice import Chalice, Response
#import sys

app = Chalice(app_name='chalice-app')


@app.route('/')
def index():

    custom_headers = {
        'Content-Type': 'text/plain',
        'X-Debug': 'some debug information'
    }

    return Response(body='hello world!',
                  status_code=200,
                  headers=custom_headers)
                    
    #print("INDEX: hello world")
    #sys.stderr.write("XX")
    #return {'hello': 'world'}
EOF



In [129]:
http 127.0.0.1:8000

HTTP/1.1 200 OK
Content-Length: 12
Content-Type: text/plain
Date: Fri, 25 Jan 2019 01:38:18 GMT
Server: BaseHTTP/0.6 Python/3.6.7
X-Debug: some debug information

hello world!



In [160]:
cat > app.py <<EOF
from chalice import Chalice

app = Chalice(app_name='helloworld')


@app.route('/')
def index():
    return {'hello': 'world'}


# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
@app.route('/hello/{name}')
def hello_name(name):
   # '/hello/james' -> {"hello": "james"}
   return {'hello': name}

@app.route('/users', methods=['POST'])
def create_user():
    # This is the JSON body the user sent in their POST request.
    user_as_json = app.current_request.json_body
    # We'll echo the json body back to the user in a 'user' key.
    return {'user': user_as_json}

EOF

In [162]:
echo '"testuser"' | http POST :8000/users

HTTP/1.1 200 OK
Content-Length: 19
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:01:46 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "user": "testuser"
}



In [163]:
echo '["testuser", {"fullname": "My full name"}]' | http POST :8000/users

HTTP/1.1 200 OK
Content-Length: 49
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:02:05 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "user": [
        "testuser",
        {
            "fullname": "My full name"
        }
    ]
}



In [164]:
cat > app.py <<EOF
from chalice import Chalice

app = Chalice(app_name='helloworld')

'''
Enabling app.debug gives much better error message for missing key than:
    { "Code": "ChaliceViewError",
      "Message": "ChaliceViewError: An internal server error occurred."
    }
'''
app.debug = True

CITIES_TO_STATE = {
    'seattle': 'WA',
    'portland': 'OR',
}


@app.route('/')
def index():
    return {'hello': 'world'}

@app.route('/cities/{city}')
def state_of_city(city):
    return {'state': CITIES_TO_STATE[city]}


EOF

In [166]:
http 127.0.0.1:8000

http 127.0.0.1:8000/cities/seattle
http :8000/cities/portland
http :8000/cities/noSuchCity


HTTP/1.1 200 OK
Content-Length: 17
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:04:26 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "hello": "world"
}

HTTP/1.1 200 OK
Content-Length: 14
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:04:26 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "state": "WA"
}

HTTP/1.1 200 OK
Content-Length: 14
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:04:26 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "state": "OR"
}

HTTP/1.1 500 Internal Server Error
Content-Length: 376
Content-Type: text/plain
Date: Fri, 25 Jan 2019 02:04:27 GMT
Server: BaseHTTP/0.6 Python/3.6.7

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chalice/app.py", line 731, in _get_view_function_response
    response = view_function(**function_args)
  File "/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app/app.py", line 25, in state_of_city
    return {'state': CITIES_TO_STATE[city]}
KeyError: 'no

In [167]:
cat > app.py <<EOF
from chalice import Chalice
from chalice import BadRequestError
from chalice import TooManyRequestsError

'''
  NOTE: Can use different error codes:
    * BadRequestError   - return a status code of 400
    * UnauthorizedError - return a status code of 401
    * ForbiddenError    - return a status code of 403
    * NotFoundError     - return a status code of 404
    * ConflictError     - return a status code of 409
    * UnprocessableEntityError - return a status code of 422
    * TooManyRequestsError - return a status code of 429
    * ChaliceViewError  - return a status code of 500
'''

app = Chalice(app_name='helloworld')

''' Enabling app.debug gives much better error message for missing key than:
    { "Code": "ChaliceViewError", "Message": "ChaliceViewError: An internal server error occurred." }
'''
app.debug = True

CITIES_TO_STATE = {
    'seattle': 'WA',
    'portland': 'OR',
}

@app.route('/')
def index():
    return {'hello': 'world'}

# BASIC: @app.route('/cities/{city}')
# BASIC: def state_of_city(city):
# BASIC:     return {'state': CITIES_TO_STATE[city]}

@app.route('/cities/{city}')
def state_of_city(city):
    try:
        return {'state': CITIES_TO_STATE[city]}
    except KeyError:
        if city == "TooMany":
            raise TooManyRequestsError("Unknown city '%s', valid choices are: %s" % ( city, ', '.join(CITIES_TO_STATE.keys())))
        raise BadRequestError("Unknown city '%s', valid choices are: %s" % ( city, ', '.join(CITIES_TO_STATE.keys())))

EOF

In [169]:
http :8000/cities/noSuchCity
http :8000/cities/TooMany


HTTP/1.1 400 Bad Request
Content-Length: 119
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:07:08 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "BadRequestError",
    "Message": "BadRequestError: Unknown city 'noSuchCity', valid choices are: seattle, portland"
}

HTTP/1.1 429 Too Many Requests
Content-Length: 126
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:07:08 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "TooManyRequestsError",
    "Message": "TooManyRequestsError: Unknown city 'TooMany', valid choices are: seattle, portland"
}



In [174]:
cat > app.py <<EOF
from chalice import Chalice
from chalice import Response
from chalice import BadRequestError
from chalice import ConflictError
from chalice import NotFoundError
from chalice import TooManyRequestsError


'''
  NOTE: Can use different error codes:
    * BadRequestError   - return a status code of 400
    * UnauthorizedError - return a status code of 401
    * ForbiddenError    - return a status code of 403
    * NotFoundError     - return a status code of 404
    * ConflictError     - return a status code of 409
    * UnprocessableEntityError - return a status code of 422
    * TooManyRequestsError - return a status code of 429
    * ChaliceViewError  - return a status code of 500
'''

import os

app = Chalice(app_name='webserver')

'''
Enabling app.debug gives much better error message for missing key than:
    { "Code": "ChaliceViewError",
      "Message": "ChaliceViewError: An internal server error occurred."
    }
'''
app.debug = True

@app.route('/')
def index():
    return os.listdir('.')

@app.route('/{item}')
def cat_file(item):
    # Get files in current dir:
    items=os.listdir('.')

    # Output contents if a file, else throw appropriate exception:
    if item in items:
        if os.path.isfile(item):
            fh = open(item)
            #return fh.readlines()
            return Response(body=" ".join(fh.readlines()),
                    status_code=200,
                    headers={'Content-Type': 'text/plain'})

        if os.path.isdir(item):
            items=os.listdir(item)
            return items

        raise ConflictError("Item '{}' exists but is not a file".format(item))

    raise NotFoundError("No such file as '{}'".format(item))

EOF

In [175]:

 http :8000/.gitignore
 http :8000/.chalice
 http :8000/.chaliceX



HTTP/1.1 200 OK
Content-Length: 38
Content-Type: text/plain
Date: Fri, 25 Jan 2019 02:09:54 GMT
Server: BaseHTTP/0.6 Python/3.6.7

.chalice/deployments/
 .chalice/venv/

HTTP/1.1 200 OK
Content-Length: 40
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:09:55 GMT
Server: BaseHTTP/0.6 Python/3.6.7

[
    "deployments",
    "config.json",
    "deployed"
]

HTTP/1.1 404 Not Found
Content-Length: 79
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:09:55 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "NotFoundError",
    "Message": "NotFoundError: No such file as '.chaliceX'"
}



In [176]:
cat > app.py <<EOF
from chalice import Chalice

app = Chalice(app_name='helloworld')

'''
    The app.current_request object also has the following properties.

    current_request.query_params - A dict of the query params for the request.
    current_request.headers - A dict of the request headers.
    current_request.uri_params - A dict of the captured URI params.
    current_request.method - The HTTP method (as a string).
    current_request.json_body - The parsed JSON body (json.loads(raw_body))
    current_request.raw_body - The raw HTTP body as bytes.
    current_request.context - A dict of additional context information
    current_request.stage_vars - Configuration for the API Gateway stage
'''


@app.route('/')
def index():
    return {'hello': 'world'}

@app.route('/inspect')
def introspect():
    return app.current_request.to_dict()

@app.route('/resource/{value}', methods=['PUT'])
def put_test(value):
    print("--> put_test('%s')".format(value))
    return {"value": value}

'''
    We can test this method using the http command:

    $ http PUT https://endpoint/api/resource/foo
    HTTP/1.1 200 OK { "value": "foo" }

    Note that the methods kwarg accepts a list of methods. Your view function will be called when any of the HTTP methods you specify are used for the specified resource. For examp
le:
'''

@app.route('/myview', methods=['POST', 'PUT', 'GET', 'DELETE'])
def myview():
    method=app.current_request.method
    #print(method)
    print("--> /myview[method={}]".format(method))
    #return {str(method), "myview"} - causes exception
    return {"method": method}

'''
The above view function will be called when either an HTTP POST or PUT is sent to /myview.

Alternatively if you do not want to share the same view function across multiple HTTP methods for the same route url, you may define separate view functions to the same route url b
ut have the view functions differ by HTTP method. For example:
'''

@app.route('/myview2', methods=['GET'])
def myview_get():
    method=app.current_request.method
    print("--> /myview2 [myview_{}]".format(method))
    #print("--> myview_get[%s]('%s')".format(request.method, value))
    return {"myview_get method": method}

@app.route('/myview2', methods=['DELETE'])
def myview_delete():
    method=app.current_request.method
    print("--> /myview2 [myview_{}]".format(method))
    #print("--> myview_delete[%s]('%s')".format(request.method, value))
    return {"myview_delete method": method}

@app.route('/myview2', methods=['POST'])
def myview_post():
    method=app.current_request.method
    print("--> /myview2 [myview_{}]".format(method))
    #print("--> myview_post[%s]('%s')".format(request.method, value))
    return {"myview_post method": method}

@app.route('/myview2', methods=['PUT'])
def myview_put():
    method=app.current_request.method
    print("--> /myview2 [myview_{}]".format(method))
    #print("--> myview_put[%s]('%s')".format(request.method, value))
    return {"myview_put method": method}


# The view function above will return {"hello": "world"}
# whenever you make an HTTP GET request to '/'.
#
# Here are a few more examples:
#
@app.route('/hello/{name}')
def hello_name(name):
   # '/hello/james' -> {"hello": "james"}
   return {'hello': name}

@app.route('/users', methods=['POST'])
def create_user():
    # This is the JSON body the user sent in their POST request.
    user_as_json = app.current_request.json_body
    # We'll echo the json body back to the user in a 'user' key.
    return {'user': user_as_json}

#
# See the README documentation for more examples.
#


EOF

In [177]:
http :8000/inspect

http ':8000/inspect?query1=value1&query2=value2' 'X-TestHeader: Foo'

http :8000/inspect

for METHOD in GET PUT POST DELETE; do
  http $METHOD :8000/myview
  http $METHOD :8000/myview2
done


HTTP/1.1 200 OK
Content-Length: 318
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:12:52 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "context": {
        "httpMethod": "GET",
        "identity": {
            "sourceIp": "127.0.0.1"
        },
        "path": "/inspect",
        "resourcePath": "/inspect"
    },
    "headers": {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate",
        "connection": "keep-alive",
        "host": "localhost:8000",
        "user-agent": "HTTPie/0.9.8"
    },
    "method": "GET",
    "query_params": null,
    "stage_vars": {},
    "uri_params": {}
}

HTTP/1.1 200 OK
Content-Length: 372
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:12:52 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "context": {
        "httpMethod": "GET",
        "identity": {
            "sourceIp": "127.0.0.1"
        },
        "path": "/inspect",
        "resourcePath": "/inspect"
    },
    "headers": {
        "accept": "*/*",
        "accept

In [178]:
cat > app.py <<EOF

import sys

from chalice import Chalice
if sys.version_info[0] == 3:
    # Python 3 imports.
    from urllib.parse import urlparse, parse_qs
else:
    # Python 2 imports.
    from urlparse import urlparse, parse_qs


app = Chalice(app_name='helloworld')


@app.route('/', methods=['POST'],
           content_types=['application/x-www-form-urlencoded'])
def index():
    parsed = parse_qs(app.current_request.raw_body.decode())
    return {
        'states': parsed.get('states', [])
    }
EOF

In [179]:
http --form POST :8000/ states=WA states=CA --debug


HTTPie 0.9.8
Requests 2.18.4
Pygments 2.3.1
Python 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
/usr/bin/python3
Linux 4.15.0-1031-aws

<Environment {
    "colors": 8,
    "config": {
        "__meta__": {
            "about": "HTTPie configuration file",
            "help": "https://httpie.org/docs#config",
            "httpie": "0.9.8"
        },
        "default_options": "[]"
    },
    "config_dir": "/home/user1/.httpie",
    "is_windows": false,
    "stderr": "<_io.TextIOWrapper name='<stderr>' mode='w' encoding='UTF-8'>",
    "stderr_isatty": true,
    "stdin": "<_io.TextIOWrapper name='<stdin>' mode='r' encoding='UTF-8'>",
    "stdin_encoding": "UTF-8",
    "stdin_isatty": true,
    "stdout": "<_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>",
    "stdout_encoding": "UTF-8",
    "stdout_isatty": true
}>

>>> requests.request(**{
    "allow_redirects": false,
    "auth": "None",
    "cert": "None",
    "data": {
        "states": "['WA', 'CA']"
    },
    "fi

In [181]:
cat > app.py <<EOF

from chalice import Chalice
from chalice import NotFoundError

app = Chalice(app_name='s3')

@app.route('/')
def index():
    return {'hello': 'world'}

import json
import boto3
from botocore.exceptions import ClientError

S3 = boto3.client('s3', region_name='us-west-2')
BUCKET = 'mjbright-uploads'

@app.route('/objects/{key}', methods=['GET', 'PUT'])
def s3objects(key):
    request = app.current_request
    if request.method == 'PUT':
        S3.put_object(Bucket=BUCKET, Key=key,
                      Body=json.dumps(request.json_body))
    elif request.method == 'GET':
        try:
            response = S3.get_object(Bucket=BUCKET, Key=key)
            return json.loads(response['Body'].read())
        except ClientError as e:
            raise NotFoundError(key)

EOF

In [186]:
aws s3 ls s3://mjbright-uploads
echo '["testuser", {"fullname": "My full name"}]' | http PUT :8000/objects/testfile
aws s3 ls s3://mjbright-uploads

2019-01-25 01:41:21        221 1
2019-01-25 01:02:13        221 hosts
2019-01-25 01:03:25        221 x
2019-01-25 01:07:47        221 y
2019-01-25 01:40:06        221 z
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 500 Internal Server Error
Content-Length: 77
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:24:32 GMT
Server: BaseHTTP/0.6 Python/3.6.7

{
    "Code": "InternalServerError",
    "Message": "An internal server error occurred."
}

2019-01-25 01:41:21        221 1
2019-01-25 01:02:13        221 hosts
2019-01-25 01:03:25        221 x
2019-01-25 01:07:47        221 y
2019-01-25 01:40:06        221 z


In [203]:
cat > app.py <<EOF

from os import environ as env

# 3rd party imports
from chalice import Chalice, Response
from twilio.rest import Client
from twilio.base.exceptions import TwilioRestException

# Twilio Config
ACCOUNT_SID = env.get('TWILIO_ACCOUNT_SID')
AUTH_TOKEN = env.get('TWILIO_AUTH_TOKEN')
FROM_NUMBER = env.get('TWILIO_FROM_NUMBER')
TO_NUMBER = env.get('TWILIO_TO_NUMBER')

app = Chalice(app_name='sms-shooter')

# Create a Twilio client using account_sid and auth token
tw_client = Client(ACCOUNT_SID, AUTH_TOKEN)

@app.route('/service/sms/send', methods=['POST'])
def send_sms():
    request_body = app.current_request.json_body
    if request_body:
        custom_headers={'X-myheader': 'cool', 'X-From': "'"+FROM_NUMBER+"'", 'X-To': "'"+TO_NUMBER+"'", 'Content-Type': 'application/json'}
        try:
            msg = tw_client.messages.create(
                from_=FROM_NUMBER,
                body=request_body['msg'],
                to=TO_NUMBER)

            if msg.sid:
                return Response(status_code=201,
                                headers=custom_headers,
                                body={'status': 'success',
                                      'data': msg.sid,
                                      'message': 'SMS <{}> successfully sent to {}'.format(request_body['msg'], TO_NUMBER)})
            else:
                return Response(status_code=200,
                                headers=custom_headers,
                                body={'status': 'failure',
                                      'message': 'Please try again!!!'})
        except TwilioRestException as exc:
            return Response(status_code=400,
                                headers=custom_headers,
                            body={'status': 'failure',
                                  'message': exc.msg})
                                  

EOF

cat > requirements.txt <<EOF
boto3==1.3.1
twilio==6.22.0
EOF

In [205]:
. ~/.twilio.rc
#http POST :8000/service/sms/send msg="MY MESSAGE" --debug
http POST :8000/service/sms/send msg="MY MESSAGE at $(date) from ${USER}@$(hostname)"

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 201 Created
Content-Length: 187
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:41:55 GMT
Server: BaseHTTP/0.6 Python/3.6.7
X-From: '+33644601324'
X-To: '+33652891534'
X-myheader: cool

{
    "data": "SMb95ef38ca6a44228a931807fb9fb78ef",
    "message": "SMS <MY MESSAGE at Fri Jan 25 02:41:54 UTC 2019 from user1@ip-172-31-21-116> successfully sent to +33652891534",
    "status": "success"
}



In [206]:
chalice deploy; chalice url

/usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Creating lambda function: chalice-app-dev
Creating Rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:568285458700:function:chalice-app-dev
  - Rest API URL: https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/
https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/


In [207]:
## BUT HOW TO PASS IN TWILIO SECRETS ?? !!

http POST https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/service/sms/send msg="MY MESSAGE at $(date) from ${USER}@$(hostname)"

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 502 Bad Gateway
Connection: keep-alive
Content-Length: 36
Content-Type: application/json
Date: Fri, 25 Jan 2019 02:43:27 GMT
Via: 1.1 c0740de9f16cf32ffedfd2f9806b8483.cloudfront.net (CloudFront)
X-Amz-Cf-Id: Fkfgd1auQ0uh9trp3WPh75YAKdyplf9YIiCdTl1Ygt7foNLJzHD6wA==
X-Cache: Error from cloudfront
x-amz-apigw-id: UCeoOHHpSK4FtUw=
x-amzn-RequestId: fd6ae314-204a-11e9-a25b-539ba9c73811

{
    "message": "Internal server error"
}



In [73]:
aws s3 mb s3://mjbright-uploads
aws s3 ls

make_bucket: mjbright-uploads
2019-01-24 06:41:23 mjbright-static-site
2019-01-24 19:44:07 mjbright-test-bucket
2019-01-25 00:56:10 mjbright-uploads


In [141]:
BUCKET_NAME='mjbright-uploads'

cat > app.py <<EOF
from chalice import Chalice
#import sys

app = Chalice(app_name="helloworld")

# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='$BUCKET_NAME')
def handler(event):
    # BUT WHERE DOES THIS OUTPUT GO?
    print("Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))
    #sys.exit(0)
EOF

In [142]:
cat app.py

from chalice import Chalice
#import sys

app = Chalice(app_name="helloworld")

# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='mjbright-uploads')
def handler(event):
    # BUT WHERE DOES THIS OUTPUT GO?
    print("Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))
    #sys.exit(0)


Deploy a local chalice server in another window using ```chalice local```, then upload a file to our bucket

In [151]:
cat > requirements.txt <<EOF
boto3==1.3.1
EOF

In [156]:
aws lambda delete-function --function-name chalice-app-dev-handler

In [157]:
cat app.py

from chalice import Chalice
#import sys

app = Chalice(app_name="helloworld")

# Whenever an object is uploaded to 'mybucket'
# this lambda function will be invoked.

@app.on_s3_event(bucket='mjbright-uploads')
def handler(event):
    # BUT WHERE DOES THIS OUTPUT GO?
    print("Object uploaded for bucket: %s, key: %s"
          % (event.bucket, event.key))
    #sys.exit(0)


In [158]:
cat requirements.txt

boto3==1.3.1


In [159]:
aws lambda list-functions

{
    "Functions": []
}


In [154]:
# Only works when deployed?
chalice deploy

Creating deployment package.
Updating policy for IAM role: chalice-app-dev
Creating lambda function: chalice-app-dev-handler
Configuring S3 events in bucket mjbright-uploads to function chalice-app-dev-handler
Deleting Rest API: lz2dkwyc3e
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chalice/awsclient.py", line 470, in delete_rest_api
    client.delete_rest_api(restApiId=rest_api_id)
  File "/usr/local/lib/python3.6/dist-packages/botocore/client.py", line 357, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/usr/local/lib/python3.6/dist-packages/botocore/client.py", line 661, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.NotFoundException: An error occurred (NotFoundException) when calling the DeleteRestApi operation: Invalid REST API identifier specified 568285458700:lz2dkwyc3e

During handling of the above exception, another exception occurred:

Traceback (most recent cal

: 2

In [143]:
aws s3 cp /etc/hosts s3://mjbright-uploads/1

upload: ../../../../../../../../etc/hosts to s3://mjbright-uploads/1


In [144]:
aws s3 ls  s3://mjbright-uploads

2019-01-25 01:41:21        221 1
2019-01-25 01:02:13        221 hosts
2019-01-25 01:03:25        221 x
2019-01-25 01:07:47        221 y
2019-01-25 01:40:06        221 z


In [88]:
ls -al /home/

total 16
drwxr-xr-x  4 root   root   4096 Jan 22 07:45 .
drwxr-xr-x 23 root   root   4096 Jan 22 07:16 ..
drwxr-xr-x  6 ubuntu ubuntu 4096 Jan 24 09:29 ubuntu
drwxr-xr-x 16 user1  user1  4096 Jan 25 00:18 user1


In [89]:
. ~/.twilio.rc

In [99]:
pwd

cat > app.py <<EOF


from os import environ as env

# 3rd party imports
from chalice import Chalice, Response
from twilio.rest import Client
from twilio.base.exceptions import TwilioRestException

# Twilio Config
ACCOUNT_SID = env.get('ACCOUNT_SID')
AUTH_TOKEN = env.get('AUTH_TOKEN')
FROM_NUMBER = env.get('FROM_NUMBER')
TO_NUMBER = env.get('TO_NUMBER')

app = Chalice(app_name='sms-shooter')

# Create a Twilio client using account_sid and auth token
tw_client = Client(ACCOUNT_SID, AUTH_TOKEN)

@app.route('/service/sms/send', methods=['POST'])
def send_sms():
    request_body = app.current_request.json_body
    return Response(status_code=201,
                                headers={'Content-Type': 'application/json'},
                                body={'status': 'success',
                                      'data': 'hello',
                                      'message': 'SMS successfully sent'})
    if request_body:
        try:
            msg = tw_client.messages.create(
                from_=FROM_NUMBER,
                body=request_body['msg'],
                to=TO_NUMBER)

            if msg.sid:
                return Response(status_code=201,
                                headers={'Content-Type': 'application/json'},
                                body={'status': 'success',
                                      'data': msg.sid,
                                      'message': 'SMS successfully sent'})
            else:
                return Response(status_code=200,
                                headers={'Content-Type': 'application/json'},
                                body={'status': 'failure',
                                      'message': 'Please try again!!!'})
        except TwilioRestException as exc:
            return Response(status_code=400,
                            headers={'Content-Type': 'application/json'},
                            body={'status': 'failure',
                                  'message': exc.msg})


EOF

/home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/chalice-app/chalice-app


In [98]:
http POST 127.0.0.1:8000/service/sms/send msg='hello'

HTTP/1.1 403 Forbidden
Content-Length: 43
Content-Type: application/json
Date: Fri, 25 Jan 2019 01:24:16 GMT
Server: BaseHTTP/0.6 Python/3.6.7
x-amzn-ErrorType: UnauthorizedException
x-amzn-RequestId: e535e69c-d749-48bb-ba93-e53b47b56f62

{
    "message": "Missing Authentication Token"
}



In [93]:
http --help

usage: http [--json] [--form] [--pretty {all,colors,format,none}]
            [--style STYLE] [--print WHAT] [--headers] [--body] [--verbose]
            [--all] [--history-print WHAT] [--stream] [--output FILE]
            [--download] [--continue]
            [--session SESSION_NAME_OR_PATH | --session-read-only SESSION_NAME_OR_PATH]
            [--auth USER[:PASS]] [--auth-type {basic,digest}]
            [--proxy PROTOCOL:PROXY_URL] [--follow]
            [--max-redirects MAX_REDIRECTS] [--timeout SECONDS]
            [--check-status] [--verify VERIFY]
            [--ssl {ssl2.3,tls1,tls1.1,tls1.2}] [--cert CERT]
            [--cert-key CERT_KEY] [--ignore-stdin] [--help] [--version]
            [--traceback] [--default-scheme DEFAULT_SCHEME] [--debug]
            [METHOD] URL [REQUEST_ITEM [REQUEST_ITEM ...]]

HTTPie - a CLI, cURL-like tool for humans. <http://httpie.org>

Positional Arguments:
  
  These arguments come after any flags and in the order they are listed here.
  Only

      
  --check-status
      By default, HTTPie exits with 0 when no network or other fatal errors
      occur. This flag instructs HTTPie to also check the HTTP status code and
      exit with an error if the status indicates one.
      
      When the server replies with a 4xx (Client Error) or 5xx (Server Error)
      status code, HTTPie exits with 4 or 5 respectively. If the response is a
      3xx (Redirect) and --follow hasn't been set, then the exit status is 3.
      Also an error message is written to stderr if stdout is redirected.
      

SSL:
  --verify VERIFY
      Set to "no" to skip checking the host's SSL certificate. You can also pass
      the path to a CA_BUNDLE file for private certs. You can also set the
      REQUESTS_CA_BUNDLE environment variable. Defaults to "yes".
      
  --ssl {ssl2.3,tls1,tls1.1,tls1.2}
      The desired protocol version to use. This will default to
      SSL v2.3 which will negotiate the highest protocol that both
      the server and you

In [ ]:
aws lambda get-function-configuration --function-name chalice-app-dev

In [ ]:
aws lambda list-versions-by-function --function-name chalice-app-dev

In [ ]:
aws lambda get-function --function-name chalice-app-dev

In [ ]:
http https://lz2dkwyc3e.execute-api.us-west-1.amazonaws.com/api/

You now should be able to access your site using a browser, or from the command-line:

In [ ]:
wget -q -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

Let's now create something a little more like a website.

We'll use the Pelican command (installed as a Python module).


In [ ]:
pelican -o PELICAN -q

cp -a PELICAN/* website/

aws s3 sync website/ s3://mjbright-static-site/

In [ ]:
wget -q -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

Unfortunately this broke our website as we have not yet set permissions for the new files we just added to our website.

Go back to the bucket list at  https://s3.console.aws.amazon.com/s3/buckets/, drill down into your website bucket.

Then select all items within the bucket and from the Access drop-down menu select "*Make Public*"

Your website should now be accessible

In [ ]:
wget -q -O - http://mjbright-static-site.s3-website-us-west-1.amazonaws.com/index.html

# More Reading

You can find more details about S3 website hosting here: https://docs.aws.amazon.com/AmazonS3/latest/dev/WebsiteHosting.html

An article describing the use of S3 for static website hosting including use of https, DNS routing
https://medium.freecodecamp.org/how-to-host-a-static-website-with-s3-cloudfront-and-route53-7cbb11d4aeea

# Cleanup

Note that we can use the ```aws s3 rm``` command to remove files from the bucket and ```aws s3 rb``` command to remove a bucket.

In [ ]:
aws s3 rm s3://mjbright-static-site --recursive
aws s3 rb s3://mjbright-static-site

It's also possible to remoce the bucket directly using the ```--force``` option:
    ```aws s3 rb --force s3://mjbright-static-site```